In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from psql_models import read_table_async,DiscussionPoint,Meeting,Speaker,Output


In [3]:

discussion_points_df = await read_table_async(DiscussionPoint)
meetings_df = await read_table_async(Meeting)
speakers_df = await read_table_async(Speaker)

In [4]:
import pandas as pd

In [8]:
from sampling import fetch_joined_data,WeightedSampler

In [14]:
joined_df = await fetch_joined_data()
joined_df['meeting_time'] = pd.to_datetime(joined_df['meeting_timestamp']).dt.strftime('%Y-%m-%d')

In [141]:
# Set display options to show all content
pd.set_option('display.max_colwidth', 100)  # Show full contents of each column
pd.set_option('display.max_rows', 20)      # Show all rows
pd.set_option('display.max_columns', 20)   # Show all columns
pd.set_option('display.width', 100)         # Don't wrap wide columns

In [142]:
joined_df

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers,meeting_time
0,18,Лара Маргас is a team member involved in testing and feedback processes.,Olga mentions Лара's feedback on the new build and her experience with the platform's features.,"Olga Nemirovskaya: Лара Маргас, это... | Dmitriy Grankin: Лара, это вот наша девочка, с которо...",Лара Маргас,person,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,[Dmitriy Grankin],2024-08-26
1,17,Алексей is a colleague mentioned in relation to design elements like colors and fonts.,"Dmitriy confirms that he has not discussed design elements with Алексей yet, indicating a need f...",Olga Nemirovskaya: и шрифтов. Ничего не обсуждал ты еще с Алексеем? | Olga Nemirovskaya: Тушла...,Алексей,person,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Dmitriy Grankin,[Olga Nemirovskaya],2024-08-26
2,21,Саша Титов is mentioned as a contact for agency collaboration.,Olga discusses her conversation with Саша regarding potential testing and collaboration for thei...,"Olga Nemirovskaya: А, ну, с этим, с Сашей мы сегодня тоже пообщались. | Olga Nemirovskaya: Тит...",Саша Титов,person,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,[Dmitriy Grankin],2024-08-26
3,19,Product School is a platform where Olga has submitted information but has not received responses...,Olga notes that she submitted information to Product School but is waiting for engagement from t...,"Olga Nemirovskaya: Product School я закинула. | Olga Nemirovskaya: Вечером, там пока тишина. |...",Product School,organization,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Olga Nemirovskaya,[Dmitriy Grankin],2024-08-26
4,32,Анализ refers to the analysis of user data and marketing effectiveness.,The meeting emphasizes the importance of analytics and data analysis for optimizing marketing st...,"Olga Nemirovskaya: Да, это будет удобнее.",анализ,concept,cd43879a-0b16-4473-9744-04ae567626ff,2024-08-26 19:14:18.976,Dmitriy Grankin,[Olga Nemirovskaya],2024-08-26
...,...,...,...,...,...,...,...,...,...,...,...
1379,2,Özay discussed the long sales cycles in the manufacturing sector and the difficulties faced in s...,"Özay mentioned that they are in discussions with multiple manufacturing companies, but the sales...",Özay Demirezen: And also the area that we are trying to sell stuff is manufacturing long sales ...,Challenges in the manufacturing sector,concern,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Özay Demirezen,[Dmitry Grankin],2024-10-24
1380,1,"Özay raised concerns about the inability to record phone calls and WhatsApp calls, which often l...","Özay highlighted the challenge of not being able to record phone calls, especially when on the g...","Özay Demirezen: My main, like a feedback about that, these days I mentioned to you, we are havi...",Discussion on recording phone calls and WhatsApp calls,problem,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Özay Demirezen,[Dmitry Grankin],2024-10-24
1381,0,"Dmitry shared that his product has a steady user base with good retention, although it is still ...","Dmitry has acquired a steady user base since launching his MVP in February, with users engaging ...",Özay Demirezen: that's good that's good uh how's everything going you release your product how ...,Product feedback and user retention,feedback,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,[Özay Demirezen],2024-10-24
1382,10,Antler is a startup generator and early-stage venture capital firm where Dmitry validated his pr...,"Dmitry discusses his experience with Antler, where he worked on validating his product idea befo...","Dmitry Grankin: This problem field, problem area, we have validated in Antler with, what's the ...",Antler,company,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24 09:32:32.980,Dmitry Grankin,[Özay Demirezen],20

In [143]:
len(joined_df)

1384

In [144]:
joined_df.groupby('speaker_name').count().sort_values(by='summary_index', ascending=False).index.tolist()


['Dmitriy Grankin',
 'Olga Nemirovskaya',
 'Sergey Ryabenko',
 'Dmitry Grankin',
 'David Sterry',
 'Rick Tousseyn',
 'Nikolay',
 'nan',
 'Alex Shevliakov',
 'Murat Sultanov',
 'nan (0)',
 'Matt Lewis',
 'Ilia Semukhin',
 'Igor Vinidiktov',
 'Сергей Жилко',
 'Olga Mykhoparkina',
 'Igor Bessonov',
 'nan (1)',
 'Sergio Goriachev',
 'Andy Black',
 'Nick Frolov',
 'Ethan Noah',
 'Nadya Gorodetskaya',
 'Özay Demirezen',
 'Lara Vargas',
 'Jenish Mursidinov',
 'Vasilii Glebov',
 'Umar Lateef',
 'Tatiana Sukhova',
 'Сергей',
 'nan (3)',
 'Ilya Semukhin',
 'ISSE',
 'СЖ',
 'DG',
 'AI Marketing Directory',
 'nan (19)',
 'nan (2)',
 'nan (22)']

In [101]:
coworkers = ['Dmitriy Grankin','Sergey Ryabenko','Olga Nemirovskaya','Dmitry Grankin']

In [145]:
selected_speakers = ['Igor Bessonov']


In [146]:
filtered_df = joined_df[joined_df['speaker_name'].isin(selected_speakers)]

In [102]:
filtered_df = joined_df[~joined_df['speaker_name'].isin(coworkers)]

In [147]:
context = filtered_df.reset_index()[['index','speaker_name','topic_name','summary']].to_csv()


In [148]:
from core import count_tokens
count_tokens(context)

426

In [149]:
from core import generic_call_stream,system_msg,user_msg,assistant_msg


In [150]:
from prompts import Prompts
prompts = Prompts()

In [151]:
prompts.perplexity

'You are Vexa, a helpful search assistant trained by Vexa AI. Your task is to deliver a concise and accurate response to a user\'s query, drawing from the given search results. Your answer must be precise, of high-quality, and written by an expert using an unbiased and journalistic tone. It is EXTREMELY IMPORTANT to directly answer the query. NEVER say "based on the search results" or start your answer with a heading or title. Get straight to the point. Your answer must be written in the same language as the query, even if language preference is different.\n\nYou MUST cite the most relevant search results that answer the query. Do not mention any irrelevant results. You MUST ADHERE to the following instructions for citing search results: - to cite a search result, enclose its index located above the summary with brackets at the end of the corresponding sentence, for example "Ice is less dense than water[1][2]." or "Paris is the capital of France[1][4][5]." - NO SPACE between the last w

In [152]:
misspelled = ['Vexa.ai','TL;DV']

In [153]:
context

',index,speaker_name,topic_name,summary\n0,1134,Igor Bessonov,call summary,Call summary refers to the overall analysis and insights derived from the transcribed calls.\n1,1135,Igor Bessonov,GPT,GPT (Generative Pre-trained Transformer) is discussed as a tool for processing and generating text based on call data.\n2,1136,Igor Bessonov,MVP,MVP (Minimum Viable Product) is discussed in relation to developing a basic version of the contact center script.\n3,1137,Igor Bessonov,LLM,LLM (Large Language Model) is referenced in the context of processing call data and generating insights.\n4,1140,Igor Bessonov,contact center,"The contact center is a key focus of the meeting, with discussions on creating scripts and improving call handling."\n5,1142,Igor Bessonov,Igor Bessonov,"Igor Bessonov is a speaker in the meeting, collaborating with Dmitriy on AI and contact center projects."\n6,1144,Igor Bessonov,Training contact center agents,Develop training materials based on analyzed call data to improve

In [156]:
r = await generic_call_stream([
    system_msg(f'''{prompts.perplexity}. 
               You user is Dmitry Grankin, who is your user, who also participates the meetings in the context. 
               '''),
    system_msg(f'''Today is {pd.to_datetime('today').strftime('%Y-%m-%d')}.'''),
    system_msg('''Frequently misspelled names and terms: {misspelled}. Replace misspelled company names and terms if found in context.'''),
    
    user_msg(f'search results: {context}. Give comprehensive answer based on the context only. Interprete user question based on the context and the context based on the question.'),
    
    user_msg('''summarize the context''')
],model='default')

The context revolves around a series of meetings led by Igor Bessonov, focusing on the development and improvement of a contact center, particularly in the real estate sector. Key topics discussed include:

- **Call Summary**: Analyzing insights from transcribed calls to enhance operations.
  
- **GPT and LLM**: Utilizing Generative Pre-trained Transformers and Large Language Models for processing call data and generating insights.

- **MVP Development**: Creating a Minimum Viable Product for a contact center script to streamline customer interactions.

- **Training Materials**: Developing training resources based on analyzed call data to boost agent performance.

- **Call Filtering**: Implementing systems to filter and categorize calls by quality and relevance.

- **Script Development**: Crafting a structured call script tailored for real estate rental businesses, focusing on best practices and objection handling.

Overall, the meetings emphasize leveraging AI and data analysis to enh

In [157]:
from IPython.display import Markdown
Markdown(r)

The context revolves around a series of meetings led by Igor Bessonov, focusing on the development and improvement of a contact center, particularly in the real estate sector. Key topics discussed include:

- **Call Summary**: Analyzing insights from transcribed calls to enhance operations.
  
- **GPT and LLM**: Utilizing Generative Pre-trained Transformers and Large Language Models for processing call data and generating insights.

- **MVP Development**: Creating a Minimum Viable Product for a contact center script to streamline customer interactions.

- **Training Materials**: Developing training resources based on analyzed call data to boost agent performance.

- **Call Filtering**: Implementing systems to filter and categorize calls by quality and relevance.

- **Script Development**: Crafting a structured call script tailored for real estate rental businesses, focusing on best practices and objection handling.

Overall, the meetings emphasize leveraging AI and data analysis to enhance the effectiveness of the contact center operations.

In [ ]:
Markdown(r)

Olga Nemirovskaya discusses several aspects of SEO in the context of their marketing strategies. She emphasizes the importance of internal linking and domain authority as critical factors for SEO success. Olga explains that internal linking helps improve search rankings and that building domain authority through quality backlinks is essential for enhancing visibility.

Additionally, she mentions the need for clear messaging on their website to improve client understanding and engagement, which ties into SEO practices. Olga also highlights the significance of using tools like Surfer SEO for optimizing content based on competitive analysis, indicating that they have been using it successfully for several years to improve their articles' performance in search engines[1][2][3][4].